# Преобразования данных о недвижимости

В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

*  удалили столбцы `index` и `Coordinates`;
*  создали признак средней площади одной комнаты `MeanRoomsSquare` и ввели коэффициент соотношения площади здания к площади участка — `AreaRatio`;
* преобразовали признак даты продажи `Date` в формат `datetime` и создали на его основе следующие столбцы: номер месяц продажи (`MonthSale`), номер дня недели продажи (`WeekdaySale`), «признак-мигалку» выходного дня (`Weekend`);
* заменили признак года постройки здания `YearBuilt` на его возраст `AgeBuilding`;
извлекли из признака адреса объекта `Address` новый признак подтипа улицы `StreetType` и удалили столбец с адресом;
уменьшили число уникальных наименований агентств по недвижимости (`SellerG`), а также число пригородов (`Suburb`);
выделили категориальные признаки и преобразовали их в тип данных `сategory`;
* заменили сокращённые названия категорий признака типа объекта `Type` на их полные названия (h — house, t —  townhouse, u — unit).

# 1.1

Преобразуйте столбец `Date` в формат `datetime` и выделите квартал (`quarter`) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [5]:
import pandas as pd

melb_df = pd.read_csv("data/melb_data_fe.csv")
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
print(melb_df.Date.dt.quarter.value_counts())

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64


# 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы `Date`, `Rooms`, `Bedroom`, `Bathroom`, `Car`.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных `category`.

In [6]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_unique_count = 150 
for col in melb_df.columns: 
    if (melb_df[col].nunique() < max_unique_count and 
        col not in cols_to_exclude): 
        melb_df[col] = melb_df[col].astype('category')
        
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

# Сортировка

`sort_values()` - метод

Параметры:

* `by` - имя столбца или список столбцов, по которым производится сортировка;

*  `axis=0` ось по которой производится сортировка (по умолчанию по строкам = 0);

* `ascending=True` - от меньшего к большему;

* `ingnore_index=False` - сохраняет исходные индексы таблицы;

* `inplace=False` - сортировать на месте (изменение исходной) или вернуть сортированную копию.

Комбинирование сортировки с фильтрацией

Найдём информацию о таунхаусах (`Type`), проданных компанией (`SellerG`) `McGrath`, у которых коэффициент соотношения площадей здания и участка (`AreaRatio`) меньше -0.8 (у которых площадь участка существенно больше площади здания). Результат отсортируем по дате продажи (`Date`) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр `ignore_index` на `True`. Для наглядности результата выберем из таблицы только столбцы `Data` и `AreaRatio`:



In [7]:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


# 2.2

Произведите сортировку столбца `AreaRatio` по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке `1558`? Ответ округлите до целого числа.



In [9]:
melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[1558, 'BuildingArea']

126.0

# 2.3

Найдите таунхаусы (`Type`) с количеством жилых комнат (`Rooms`) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (`MeanRoomsSquare`). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [16]:
mask1 = melb_df.Type == 'townhouse'
mask2 = melb_df.Rooms > 2
melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price']

1300000.0